In [1]:
import sys
sys.path.append("../module/")
sys.path.append("../learning/")

import pickle
import menconn
from typelinking import *
import time
import marisa_trie
import pickle
from dataset import *
from wikidata_linker_utils.offset_array import OffsetArray
import train_type as tp
from collections import defaultdict
from tqdm import tqdm
import re
from functools import partial
from tqdm import tqdm_notebook
import os
from nltk import ngrams
from nltk.corpus import stopwords

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dataroot = '..'
tagger = tp.SequenceTagger(os.path.join(dataroot,'ja_model/'))
type_oracle = load_oracle_classification(os.path.join(dataroot, "data/classifications/type_classification"))
trie_index2indices_values, trie_index2indices_counts, trie = load_trie(os.path.join(dataroot, 'data/ja_trie'))
with open(os.path.join(dataroot, 'data/wikidata/indicies2wikititle.pkl'), 'rb') as hdl:
    indices2title = pickle.load(hdl)

INFO:tensorflow:Restoring parameters from ../ja_model/model.ckpt


In [25]:
import MeCab
start = time.time()
mtag = MeCab.Tagger('-Owakati')
sentence = """
バラク・オバマは基本的に言ってインテリ層に人気のある黒人だったが、
ドナルド・トランプは白人主義者や陰謀論者から人気を集めている。
"""
sentence_parsed = mtag.parse(sentence)
ts = []
sep = ''
for n in range(1, 10):
    n_grams = ngrams(sentence_parsed.split(), n)
    for grams in n_grams:
        ts.append(sep.join(list(grams)))
ts = [t for t in ts if trie.get(t) is not None]
end = time.time()
print('Time:{}'.format(end-start))

Time:0.001161336898803711


In [26]:
import time
start = time.time()
results = None
tokenize = partial(menconn.ja_tokenize, ts=ts)
sent_splits, model_probs = solve_model_probs(sentence, tagger, tokenize=tokenize)
entities = run(ts, sent_splits, model_probs, indices2title, type_oracle, trie, trie_index2indices_values, trie_index2indices_counts)
preds = []
for entity in entities:
    if entity is not None:
        preds.append(entity['ja'])
    else:
        preds.append(None)
results = [{'mention':x, 'pred': y} for x,y in zip(ts,preds)]
end = time.time()
print("Time:{}".format(end-start))

Time:0.019841432571411133


In [27]:
results

[{'mention': 'バラク', 'pred': 'バラク (チャガタイ家)'},
 {'mention': '・', 'pred': '中黒'},
 {'mention': 'オバマ', 'pred': 'バラク・オバマ'},
 {'mention': 'は', 'pred': 'は'},
 {'mention': '基本', 'pred': '基本'},
 {'mention': '的', 'pred': '的'},
 {'mention': 'に', 'pred': '日本の鉄道駅一覧 に'},
 {'mention': 'て', 'pred': 'て'},
 {'mention': 'インテリ', 'pred': 'インテリ'},
 {'mention': '層', 'pred': '層 (数学)'},
 {'mention': 'に', 'pred': '日本の鉄道駅一覧 に'},
 {'mention': '人気', 'pred': '人気'},
 {'mention': 'の', 'pred': 'の'},
 {'mention': 'ある', 'pred': '存在記号'},
 {'mention': '黒人', 'pred': '黒人'},
 {'mention': 'た', 'pred': 'た'},
 {'mention': 'が', 'pred': 'が'},
 {'mention': '、', 'pred': '読点'},
 {'mention': 'ドナルド', 'pred': 'ドナルドダック'},
 {'mention': '・', 'pred': '中黒'},
 {'mention': 'トランプ', 'pred': 'トランプ'},
 {'mention': 'は', 'pred': 'は'},
 {'mention': '白人', 'pred': '白人'},
 {'mention': '主義', 'pred': '主義'},
 {'mention': '者', 'pred': '個人'},
 {'mention': 'や', 'pred': 'や'},
 {'mention': '陰謀', 'pred': '陰謀'},
 {'mention': '論', 'pred': '論'},
 {'mention': '者', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True